In [1]:
import pandas as pd
# Membaca file CSV
data = pd.read_csv('/content/sample_data/dataset-siswasmkn6-setelah-diolah.csv')

In [2]:
data.head()

,Jenis Tinggal,Alat Transportasi,Penerima KPS,Pekerjaan Ayah,Penghasilan Ayah,Pekerjaan Ibu,Penghasilan Ibu,Penerima KIP,Layak PIP (usulan dari sekolah),Kebutuhan Khusus
0,1,1,1,1,3.0,5,1,1,1,0
1,1,1,0,2,4.0,2,4,0,0,0
2,1,2,0,2,5.0,2,5,0,1,0
3,1,2,0,2,5.0,5,1,0,0,0
4,1,2,0,2,5.0,5,1,0,0,0


In [22]:
xdata = data.drop('Layak PIP (usulan dari sekolah)', axis=1)
ydata = data['Layak PIP (usulan dari sekolah)']

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics

In [25]:
xtrain, xtest, ytrain, ytest = train_test_split(xdata, ydata, test_size=0.4, random_state=1)

In [26]:
nb_clf = GaussianNB()
nb_clf.fit(xtrain, ytrain)

GaussianNB()

In [27]:
y_pred = nb_clf.predict(xtest)

In [28]:
accuracy = metrics.accuracy_score(ytest, y_pred)
print("Akurasi:", accuracy * 100, "%")

Akurasi: 74.4 %


In [29]:
import pickle
pickle_out = open("nb_clf.pkl", "wb")
pickle.dump(nb_clf, pickle_out)
pickle_out.close()

In [30]:
!pip install streamlit

In [32]:
%%writefile app.py
import pandas as pd
import numpy as np
import pickle
import streamlit as st
from PIL import Image

# Load model yang digunakan untuk melakukan prediksi
pickle_in = open('nb_clf.pkl', 'rb')
nb_clf = pickle.load(pickle_in)

# Mapping pekerjaan sesuai dengan kategori yang diberikan
job_mapping = {
    0: 'Pedagang Kecil', 1: 'Karyawan Swasta', 2: 'Wiraswasta', 3: 'PNS/TNI/Polri',
    4: 'Lainnya', 5: 'Tidak bekerja', 6: 'Wirausaha', 7: 'Buruh', 8: 'Sudah Meninggal',
    9: 'Tidak Diketahui', 10: 'Pensiunan', 11: 'Petani', 12: 'Pedagang Besar',
    13: 'Tidak dapat diterapkan', 14: 'Peternak'
}

# Mapping hasil prediksi menjadi status layak PIP
prediction_mapping = {0: 'Tidak Layak', 1: 'Layak'}

def welcome():
    return 'Welcome'

# mendefinisikan fungsi prediction yang akan memproses data inputan dari user
def prediction(jenis_tinggal, alat_transportasi, penerima_kps, pekerjaan_ayah, penghasilan_ayah, pekerjaan_ibu, penghasilan_ibu, penerima_kip, kebutuhan_khusus):
    input_data = [[jenis_tinggal, alat_transportasi, penerima_kps, pekerjaan_ayah, penghasilan_ayah, pekerjaan_ibu, penghasilan_ibu, penerima_kip, kebutuhan_khusus]]
    prediction = nb_clf.predict(input_data)
    return prediction_mapping[prediction[0]]

# main function untuk web
def main():
    st.title('🌟Prediksi Beasiswa🌟')

    # menentukan style web, background
    html_temp = """
    <div style="background-color:#007BFF; padding:20px; border-radius:10px; box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);">
          <h2 style="color:white; text-align:center; font-family: 'Arial', sans-serif;">Catatan Penting:</h2>
          <p style="color:white; text-align:center; font-size: 16px; font-family: 'Arial', sans-serif;">
              (Aplikasi ini untuk prediksi pemilihan siswa tidak mampu agar mendapatkan beasiswa dari pihak sekolah dan direkomendasikan pada program pemerintah).
          </p>
          <ul style="color:white; font-family: 'Arial', sans-serif;">
              <li>Pilih kategori menu dari dropdown.</li>
              <li>Masukkan angka dengan asumsi yang telah ditentukan di bawah ini:</li>
          </ul>
          <div style="background-color:white; color:black; padding:15px; border-radius:10px; font-family: 'Arial', sans-serif;">
              <h4 style="color:black;">Mapping Pekerjaan Ayah/Ibu:</h4>
              <ul>
                  <li>Pedagang Kecil = 1</li>
                  <li>Karyawan Swasta = 2</li>
                  <li>Wiraswasta = 3</li>
                  <li>PNS/TNI/Polri = 4</li>
                  <li>Tidak bekerja = 5</li>
                  <li>Wirausaha = 6</li>
                  <li>Buruh = 7</li>
                  <li>Sudah Meninggal = 8</li>
                  <li>Tidak Diketahui = 9</li>
                  <li>Pensiunan = 10</li>
                  <li>Petani = 11</li>
                  <li>Pedagang Besar = 12</li>
                  <li>Tidak dapat diterapkan = 13</li>
                  <li>Peternak = 14</li>
                  <li>Lainnya = 0</li>
              </ul>
              <h4 style="color:black;">Mapping Penghasilan Ibu/Ayah:</h4>
              <ul>
                  <li>Tidak Berpenghasilan = 1</li>
                  <li>Kurang dari Rp. 500,000 = 2</li>
                  <li>Rp. 500,000 - Rp. 999,999 = 3</li>
                  <li>Rp. 1,000,000 - Rp. 1,999,999 = 4</li>
                  <li>Rp. 2,000,000 - Rp. 4,999,999 = 5</li>
                  <li>Rp. 5,000,000 - Rp. 20,000,000 = 6</li>
              </ul>
              <h4 style="color:black;">Mapping Penerima KPS/KIP/PIP:</h4>
              <ul>
                  <li>Ya = 1</li>
                  <li>Tidak = 0</li>
              </ul>
              <h4 style="color:black;">Mapping Jenis Tinggal:</h4>
              <ul>
                  <li>Bersama Orang Tua = 1</li>
                  <li>Wali = 2</li>
                  <li>Kost = 3</li>
                  <li>Pesantren = 4</li>
                  <li>Asrama = 5</li>
                  <li>Panti Asuhan = 6</li>
                  <li>Lainnya = 0</li>
              </ul>
              <h4 style="color:black;">Mapping Kebutuhan Khusus:</h4>
              <ul>
                  <li>K - Kesulitan Belajar = 1</li>
                  <li>C - Tuna grahita ringan = 2</li>
                  <li>C1 - Tuna grahita sedang = 3</li>
                  <li>C1, K = 4</li>
                  <li>B, K = 5</li>
                  <li>B - Tuna rungu = 6</li>
                  <li>Tidak ada = 0</li>
              </ul>
              <h4 style="color:black;">Mapping Alat Transportasi:</h4>
              <ul>
                  <li>Sepeda = 1</li>
                  <li>Sepeda Motor = 2</li>
                  <li>Jalan Kaki = 3</li>
                  <li>Mobil Pribadi = 4</li>
                  <li>Mobil/Bus Antar Jemput = 5</li>
                  <li>Andong/Bendi/Sado/Dokar/Becak = 6</li>
                  <li>Ojek = 7</li>
                  <li>Angkutan Umum/Bus/Pete-Pete = 8</li>
                  <li>Lainnya = 0</li>
              </ul>
          </div>
      </div>
    """
    st.markdown(html_temp, unsafe_allow_html=True)

    # membuat inputan angka untuk pekerjaan ayah dan ibu
    jenis_tinggal = st.text_input('Jenis Tinggal', '0')
    alat_transportasi = st.text_input('Alat Transportasi', '0')
    penerima_kps = st.text_input('Penerima KPS', '0')

    # Input pekerjaan dalam bentuk angka (sesuai mapping)
    pekerjaan_ayah = st.selectbox('Pekerjaan Ayah (0-14)', list(job_mapping.keys()))
    penghasilan_ayah = st.text_input('Penghasilan Ayah', '0')

    pekerjaan_ibu = st.selectbox('Pekerjaan Ibu (0-14)', list(job_mapping.keys()))
    penghasilan_ibu = st.text_input('Penghasilan Ibu', '0')

    penerima_kip = st.text_input('Penerima KIP', '0')
    kebutuhan_khusus = st.text_input('Kebutuhan Khusus', '0')

    result = ''

    # jika tombol prediksi ditekan
    if st.button('🔍 Prediksi'):
        try:
            # Konversi input ke tipe numerik yang sesuai
            jenis_tinggal = int(jenis_tinggal)
            alat_transportasi = int(alat_transportasi)
            penerima_kps = int(penerima_kps)
            penghasilan_ayah = int(penghasilan_ayah)
            penghasilan_ibu = int(penghasilan_ibu)
            penerima_kip = int(penerima_kip)
            kebutuhan_khusus = int(kebutuhan_khusus)

            # Prediksi hasil
            result = prediction(jenis_tinggal, alat_transportasi, penerima_kps, pekerjaan_ayah, penghasilan_ayah, pekerjaan_ibu, penghasilan_ibu, penerima_kip, kebutuhan_khusus)
            st.success(f'Hasil Prediksi adalah: {result}')
        except ValueError:
            st.error('Harap masukkan nilai yang valid!')

if __name__ == '__main__':
    main()

Overwriting app.py


In [33]:
!wget -q -O - ipv4.icanhazip.com

35.194.95.251


In [ ]:
!streamlit run app.py & npx localtunnel --port 8501



⠙⠹
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.194.95.251:8501

⠸⠼⠴your url is: https://fluffy-hats-work.loca.lt
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
